In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

In [2]:
#fetching WHO COVID FAQ

links=['https://www.who.int/news-room/questions-and-answers/item/coronavirus-disease-(covid-19)-vaccines',
     'https://www.who.int/news-room/questions-and-answers/item/coronavirus-disease-(covid-19)-vaccines-safety',
     'https://www.who.int/news-room/questions-and-answers/item/coronavirus-disease-(covid-19)-covid-19-vaccines-and-people-living-with-hiv',
     'https://www.who.int/news-room/questions-and-answers/item/coronavirus-disease-use-of-emergency-use-listing-procedure-forvaccines-against-covid-19']
q=[]
a=[]
for link in links:
    source=requests.get(link)
    soup=BeautifulSoup(source.text,'html.parser')
    c=soup.find_all('a',{'class':'sf-accordion__link'})
    for i in c:
        q.append(' '.join(i.text.split()))
    c=soup.find_all('p',{'class':'sf-accordion__summary'})
    for i in c:
        a.append(' '.join(i.text.split()))
df=pd.DataFrame()
df['Question']=q
df['Answer']=a
df.head()

,Question,Answer
0,What vaccines are there against COVID-19?,There are several COVID-19 vaccines validated ...
1,What are the benefits of getting vaccinated?,Getting vaccinated could save your life. COVID...
2,Who should get vaccinated against COVID-19?,WHO recommends that while vaccine supply is li...
3,Who should not be vaccinated against COVID-19?,There are very few conditions that would exclu...
4,Should I be vaccinated if I have had COVID-19?,"Yes. Even if you have already had COVID-19, yo..."


In [3]:
import numpy as np
import nltk
import string

In [4]:
df['Question']=df['Question'].str.lower()

In [5]:
def remove_punctuation(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

In [6]:
df['Question_']=df['Question'].apply(remove_punctuation)

In [7]:
df.head()

,Question,Answer,Question_
0,what vaccines are there against covid-19?,There are several COVID-19 vaccines validated ...,what vaccines are there against covid19
1,what are the benefits of getting vaccinated?,Getting vaccinated could save your life. COVID...,what are the benefits of getting vaccinated
2,who should get vaccinated against covid-19?,WHO recommends that while vaccine supply is li...,who should get vaccinated against covid19
3,who should not be vaccinated against covid-19?,There are very few conditions that would exclu...,who should not be vaccinated against covid19
4,should i be vaccinated if i have had covid-19?,"Yes. Even if you have already had COVID-19, yo...",should i be vaccinated if i have had covid19


In [8]:
df['Question_']=df['Question_'].apply(nltk.word_tokenize)

In [9]:
lemmer=nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [10]:
df['Question_']=df['Question_'].apply(LemTokens)

In [11]:
df.head()

,Question,Answer,Question_
0,what vaccines are there against covid-19?,There are several COVID-19 vaccines validated ...,"[what, vaccine, are, there, against, covid19]"
1,what are the benefits of getting vaccinated?,Getting vaccinated could save your life. COVID...,"[what, are, the, benefit, of, getting, vaccina..."
2,who should get vaccinated against covid-19?,WHO recommends that while vaccine supply is li...,"[who, should, get, vaccinated, against, covid19]"
3,who should not be vaccinated against covid-19?,There are very few conditions that would exclu...,"[who, should, not, be, vaccinated, against, co..."
4,should i be vaccinated if i have had covid-19?,"Yes. Even if you have already had COVID-19, yo...","[should, i, be, vaccinated, if, i, have, had, ..."


In [12]:
def join_lemmer_token(token):
    return ' '.join(token)

In [13]:
df['Question_']=df['Question_'].apply(join_lemmer_token)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
df.head()

,Question,Answer,Question_
0,what vaccines are there against covid-19?,There are several COVID-19 vaccines validated ...,what vaccine are there against covid19
1,what are the benefits of getting vaccinated?,Getting vaccinated could save your life. COVID...,what are the benefit of getting vaccinated
2,who should get vaccinated against covid-19?,WHO recommends that while vaccine supply is li...,who should get vaccinated against covid19
3,who should not be vaccinated against covid-19?,There are very few conditions that would exclu...,who should not be vaccinated against covid19
4,should i be vaccinated if i have had covid-19?,"Yes. Even if you have already had COVID-19, yo...",should i be vaccinated if i have had covid19


In [16]:
greet_input=('hello','hi','greetings','sup',"what's up",'hey')
greet_response=('Hi, do you need some assistance?','Hey, thanks for your visit, can we help you in any way?','Hello! How can I help you today?')

In [17]:
def max_cosine_similarity(text):
    m=0
    m_index=0
    for i in range(len(df)):
        vectorizer = CountVectorizer()
        bow_matrix = vectorizer.fit_transform([text,df['Question_'][i]])
        bow_matrix.toarray()
        if cosine_similarity(bow_matrix[0],bow_matrix[1])>m:
            m=cosine_similarity(bow_matrix[0],bow_matrix[1])
            m_index=i
    if m>0.5:
        return df['Answer'][m_index]
    return "I'm Sorry! I don't understand you"

In [19]:
flag= True
print("BOT : Hi, thanks for visiting COVID 19 Vaccination Q&A, can we help you in any way?"+'\n\t'+'If you want to exit any time, just type Bye')
while flag==True:
    user_response=input('User : ').lower()
#     if user_response=='thanks' or user_response=='thanks you' or user_response=='bye':
    if 'thank' in user_response or 'bye' in user_response:
        flag=False
        print('BOT : Thanks for visiting Good Bye! Take Care')
    elif user_response in greet_input:
        print('BOT : '+random.choice(greet_response))
    else:
        print(max_cosine_similarity(user_response))

BOT : Hi, thanks for visiting COVID 19 Vaccination Q&A, can we help you in any way?
	If you want to exit any time, just type Bye
User : side effect of covid19
Like with any vaccine, some people will experience mild to moderate side effects after being vaccinated against COVID-19. This is a normal sign that the body is developing protection. Side effects to COVID-19 vaccines include a fever, tiredness, headache, muscle ache, chills, diarrhoea and pain or redness at the injection site. Not everyone will experience side effects. Most side effects go away within a few days on their own. You can manage any side effects with rest, plenty of non-alcoholic liquids and taking medication to manage pain and fever, if needed. If you are worried that the side effects that you are experiencing are unusual, if the pain in the arm where you got the injection gets worse after 24 hours or your side effects don’t go away in a few days, contact your healthcare provider for advice. More serious or long-las